In [ ]:
import psycopg2
import os
from dotenv import load_dotenv
from psycopg2.extras import RealDictCursor
from datetime import datetime, timedelta
from app import convert_date_to_week_format, get_db_connection, process_shipping_plans_carry_over, process_purchase_orders_carry_over

# .env 파일에서 환경 변수 로드
load_dotenv()

In [ ]:
supplier_filter = 'MOKA'
to_site_filter = 'ECHK'
week_from = convert_date_to_week_format(datetime.now())
week_to = convert_date_to_week_format(datetime.now() + timedelta(weeks=4))

In [ ]:
sp_conditions = ["is_deleted = FALSE"]
sp_params = []

In [ ]:
if supplier_filter:
    sp_conditions.append("from_site = %s")
    sp_params.append(supplier_filter)
if to_site_filter:
    sp_conditions.append("to_site ILIKE %s")
    sp_params.append(f"%{to_site_filter}%")
if week_from and week_to:
    sp_conditions.append("""(
        (shipping_week >= %s AND shipping_week <= %s AND is_finished = FALSE)
        OR (shipping_week < %s AND is_finished = FALSE)
    )""")
    sp_params.extend([week_from, week_to, week_from])
else:
    sp_conditions.append("is_finished = FALSE")
sp_where_clause = " AND ".join(sp_conditions)

In [ ]:
conn = get_db_connection()
cursor = conn.cursor()
cursor.execute(f'''
            SELECT * FROM shipping_plans
            WHERE {sp_where_clause}
            ORDER BY id DESC
        ''', sp_params)
plans = cursor.fetchall()

In [ ]:
po_conditions = ["status = 'Active'"]
po_params = []
if supplier_filter:
    po_conditions.append("from_site = %s")
    po_params.append(supplier_filter)
if to_site_filter:
    po_conditions.append("to_site ILIKE %s")
    po_params.append(f"%{to_site_filter}%")
week_from_date = week_from[:10] if week_from else ''
week_to_date = week_to[:10] if week_to else ''
if week_from and week_to:
    po_conditions.append("""(
        (rsd >= %s AND rsd <= %s AND is_finished = FALSE)
        OR (rsd < %s AND is_finished = FALSE)
    )""")
    po_params.extend([week_from_date, week_to_date, week_from_date])
elif week_from:
    po_conditions.append("""(
        rsd >= %s AND is_finished = FALSE
        OR (rsd < %s AND is_finished = FALSE)
    )""")
    po_params.extend([week_from_date, week_from_date])
elif week_to:
    po_conditions.append("rsd <= %s AND is_finished = FALSE")
    po_params.append(week_to_date)
else:
    po_conditions.append("is_finished = FALSE")
po_where_clause = " AND ".join(po_conditions)
cursor.execute(f'''
    SELECT id, po_number, from_site, to_site, model, po_qty, status, rsd, shipped_quantity, is_finished, remark FROM purchase_orders
    WHERE {po_where_clause}
    ORDER BY id DESC
''', po_params)
po_rows = cursor.fetchall()

In [ ]:
default_weekname = convert_date_to_week_format(datetime.now())

In [ ]:
# Carry over 처리
plans = process_shipping_plans_carry_over(plans, default_weekname)
po_rows = process_purchase_orders_carry_over(po_rows, default_weekname)

In [ ]:
po_map = {}
for po in po_rows:
    weekname = convert_date_to_week_format(po['rsd']) if po['rsd'] else ''
    key = (po['from_site'], po['to_site'], po['model'], weekname)
    po_map[key] = po

In [ ]:
# PO 딕셔너리 생성
po_map = {}
for po in po_rows:
    key = (po['from_site'], po['to_site'], po['model'], po['shipping_week'])
    if key not in po_map:
        po_map[key] = []
    po_map[key].append({'po_number': po['po_number'], 'po_qty': po['po_qty']})

# 선적계획에 PO 매칭
for plan in plans:
    key = (plan['from_site'], plan['to_site'], plan['model_name'], plan['shipping_week'])
    plan['matched_pos'] = po_map.get(key, [])

In [ ]:
plans